In [5]:
import numpy as np
import ctypes
import pyautogui

from PIL import Image, ImageDraw
import decimal

from pytesseract import pytesseract
import pandas as pd
from tqdm import tqdm
import time
import mss
import mss.tools
from screeninfo import get_monitors

In [6]:
def get_screen():
    img = None
    with mss.mss() as sct:
        # Get information of monitor 2
        monitor_number = 2
        mon = sct.monitors[monitor_number]

        # The screen part to capture
        monitor = {
            "top": mon["top"],
            "left": mon["left"],
            "width": mon["width"],
            "height": mon["height"],
            "mon": monitor_number,
        }
        output = "sct-mon{mon}_{top}x{left}_{width}x{height}.png".format(**monitor)

        # Grab the data
        img = np.array(sct.grab(monitor))
        img = Image.fromarray(img)
        return img

def get_cargo_box():
    return [372, 638, 630, 643] #x,y Top Left ,x,y Bottom Right

def convert_to_baw(img):
    thresh = 20
    fn = lambda x : 255 if x > thresh else 0
    img = img.convert('L').point(fn, mode='1')
    return img


In [7]:
full_screen_image = get_screen()
img = convert_to_baw(full_screen_image.crop(get_cargo_box()))
img

In [8]:
img_array = np.array(img)
percent_full = len(img_array[img_array==True]) / (len(img_array[img_array==True]) + len(img_array[img_array==False]))
print(f'Percent Cargo Full {percent_full: .2f}')

Percent Cargo Full  0.10
